In [1]:
#https://www.kaggle.com/anikannal/solar-power-generation-data/tasks

In [30]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics import median_absolute_error

# Categoricals
- One Hot Encoding
- Count/Frequency
- Target Encoding ("Model" encoding)
- Embedding
- Ordinal?

In [54]:
train = pd.read_parquet("train.parquet")
val = pd.read_parquet("val.parquet")

In [55]:
train2 = train.copy()
train2['WEEKDAY'] = train2['DATE_TIME'].dt.weekday
train2['HOUR'] = train2['DATE_TIME'].dt.hour
train2['MINUTE'] = train2['DATE_TIME'].dt.minute


val2 = val.copy()
val2['WEEKDAY'] = val2['DATE_TIME'].dt.weekday
val2['HOUR'] = val2['DATE_TIME'].dt.hour
val2['MINUTE'] = val2['DATE_TIME'].dt.minute
cats = ['SOURCE_KEY', 'WEEKDAY','HOUR', 'MINUTE']


# Category Encoders - Count/Frequency Encoding

In [57]:
from category_encoders import CountEncoder

#['vermelho', 'verde', 'amarelo', 'verde'] -> {"vermelho": 1/4, "verde": 2/4, "amarelo": 1/4}

In [58]:
for c in cats:
    cenc = CountEncoder(normalize=False)
    train2['COUNT_{}'.format(c)] = cenc.fit_transform(train2[c])
    val2['COUNT_{}'.format(c)] = cenc.transform(val2[c])

In [59]:
from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

Xtr, ytr = train2.filter(regex=r"COUNT"), train2['Y4WIN']
mdl.fit(Xtr,ytr)

p = mdl.predict(val2.filter(regex=r"COUNT"))
median_absolute_error(val2['Y4'], p)

2.616387020306478

# Category Encoders - Target Encoding

In [63]:
from category_encoders import TargetEncoder

#['vermelho': 1, 'verde': 0, 'amarelo':0, 'verde': 1] -> {"vermelho": 1/1 = 0, "verde": (0+1)/2 = 0.5, "amarelo": 0/1 = 0}

In [64]:
for c in cats:
    tenc = TargetEncoder()
    train2['TGT_{}'.format(c)] = tenc.fit_transform(train2[c], train2['Y4WIN'])
    val2['TGT_{}'.format(c)] = tenc.transform(val2[c])

/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/ubuntu/anaconda3/envs/rapids_p37/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [65]:
from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

Xtr, ytr = train2.filter(regex=r"TGT"), train2['Y4WIN']
mdl.fit(Xtr,ytr)

p = mdl.predict(val2.filter(regex=r"TGT"))
median_absolute_error(val2['Y4'], p)

2.6406194739607116

# RAPIDS Target Encoding

In [74]:
from cuml.preprocessing import TargetEncoder

In [75]:
for c in cats:
    tenc = TargetEncoder(n_folds=2)
    train2['TGT_{}'.format(c)] = tenc.fit_transform(train2[c], train2['Y4WIN'])
    val2['TGT_{}'.format(c)] = tenc.transform(val2[c])
    


In [76]:
from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

Xtr, ytr = train2.filter(regex=r"TGT"), train2['Y4WIN']
mdl.fit(Xtr,ytr)

p = mdl.predict(val2.filter(regex=r"TGT"))
median_absolute_error(val2['Y4'], p)

4.786011839166587

# RAPIDS KNN

In [77]:
from cuml.neighbors import KNeighborsRegressor
mdl = KNeighborsRegressor(n_neighbors=1, metric='manhattan')

Xtr, ytr = train2.filter(regex=r"TGT"), train2['Y4WIN']
mdl.fit(Xtr,ytr)

p = mdl.predict(val2.filter(regex=r"TGT"))
median_absolute_error(val2['Y4'], p)

3.7589263916015625

# RAPIDS OHE + KNN

In [78]:
from cuml.preprocessing import OneHotEncoder
import cudf
import cupy

cudf_train2 = cudf.from_pandas(train2[cats])
cudf_val2 = cudf.from_pandas(val2[cats])

ohenc = OneHotEncoder(handle_unknown='ignore', sparse=False)
cudf_train2 = ohenc.fit_transform(cudf_train2)
cudf_val2 = ohenc.transform(cudf_val2)

In [79]:
from cuml.neighbors import KNeighborsRegressor
mdl = KNeighborsRegressor(n_neighbors=1, metric='euclidean')

Xtr, ytr = cudf_train2, train2['Y4WIN']
mdl.fit(Xtr,ytr)

p = mdl.predict(cudf_val2)
p = cupy.asnumpy(p)
median_absolute_error(val2['Y4'], p)

2.0357141672356036